# ConsGenIncProcessModel.py


In [ ]:
# Initial imports and notebook setup
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
from copy import copy


from HARK.ConsumptionSaving.ConsGenIncProcessModel import *
import HARK.ConsumptionSaving.ConsumerParameters as Params

from HARK.utilities import plotFuncs

This module contains models to solve consumption-saving models with idiosyncratic shocks to income in which shocks are not necessarily fully transitory nor fully permanent. Moreover, next period's expected permanent or persistent income level $p_{t+1}$ is not necessarily a linear function of $p_t$.  That is, this module contains models that extends $\texttt{ConsIndShockModel}$ by **generalizing the income process**.  As a result, the models in this module explicitly track persistent income $p_t$ as a state variable, rather than normalizing it out as in $\texttt{ConsIndShockModel}$.

This module currently includes three models:
1. A model with a generic, unspecified persistent income transition rule.
2. A model that specifies the persistent income rule in model (1) as a linear function, with fully permanent shocks.
3. A model that specifies the persistent income rule in model (1) as an AR(1) in logs.

## GenIncProcessConsumerType

This model allows consumers to have a generic function $G_{t+1}$ that translates current persistent income $p_t$ into expected next period persistent income $p_{t+1}$.  The user can specify any function $G$ that s/he wants.  The default behavior is that a new instance of $\texttt{GenIncProcessConsumerType}$ has **no** dynamics in persistent income: $G_{t+1}(p_t) = p_t$.  Any non-trivial income dynamics must be specified by hand by the user, or constructed by a method in a subclass of $\texttt{GenIncProcessConsumerType}$ (see below).


The agent's problem can be written in Bellman form as:

\begin{eqnarray*}
v_t(M_t,p_t) &=& \max_{C_t} U(C_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [v_{t+1}(M_{t+1}, p_{t+1}) ], \\
A_t &=& M_t - C_t, \\
A_t &\geq& \underline{a} p_t, \\
M_{t+1} &=& R A_t + \theta_{t+1}, \\
p_{t+1} &=& G_{t+1}(p_t)\psi_{t+1}, \\
\psi_{t+1},\theta_{t+1} \sim F_{\psi \theta,t+1}, &\qquad& \mathbb{E} [F_{\psi t}] = 1, \\
U(C) &=& \frac{C^{1-\rho}}{1-\rho}.
\end{eqnarray*}

The one period problem for this model is solved by the function $\texttt{solveConsGenIncProcess}$, which creates an instance of the class $\texttt{ConsGenIncProcessSolver}$. The class $\texttt{GenIncProcessConsumerType}$ extends $\texttt{IndShockConsumerType}$ to represents agents in this model. To construct an instance of this class, several  parameters must be passed to the constructor as shown in the table below (parameters can be either "primitive" if they are directly specified by the user or "constructed" if they are built by a class method using simple parameters specified by the user).

### Example parameter values to solve an instance of GenIncProcessConsumerType

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
| $\beta$ |Intertemporal discount factor  | $\texttt{DiscFac}$ | 0.96 | |
| $\rho $ |Coefficient of relative risk aversion | $\texttt{CRRA}$ | 2.0 | |
| $R $ | Risk free interest factor | $\texttt{Rfree}$ | 1.03 | |
| $1 - \mathsf{D}$ |Survival probability | $\texttt{LivPrb}$ | [0.98] | |
| $\underline{a} $ |Artificial borrowing constraint, as asset-to-persistent income ratio | $\texttt{BoroCnstArt}$ | 0.0 | | 
| $(none) $ |Indicator of whether $\texttt{vFunc}$ should be computed | $\texttt{vFuncBool}$ | 'True' | |
| $(none)$ |Indicator of whether $\texttt{cFunc}$ should use cubic lines | $\texttt{CubicBool}$ | 'False' |  |
|$F $ |A list containing three arrays of floats, representing a discrete <br> approximation to the income process: <br>event probabilities, persistent shocks, transitory shocks | $\texttt{IncomeDstn}$ | - |$\surd$ |
| $G$ |Expected persistent income next period as a function of current persistent income | $\texttt{pLvlNextFunc}$ | - | $\surd$ |
| $ (none) $ |Grid of persistent income levels | $\texttt{pLvlGrid}$ | - |$\surd$ |
| $ (none) $ | Grid of end-of-period assets above minimum | $\texttt{aXtraGrid}$ | - |$\surd$ |

### Constructed inputs to solve GenIncProcess
The "constructed" inputs are automatically built when a new instance is created, using the class methods described below.  Each method expects certain parameters to have been passed to the class constructor method (and thus assigned as attributes of this instance), usually as an entry in a dictionary.  These methods are described below.


* The input $\texttt{IncomeDstn}$ is created by the method $\texttt{updateIncomeProcess}$ which inherits from $\texttt{IndShockConsumerType}$. (*hyperlink to that noteboook*)


* The input $\texttt{pLvlNextFunc}$ is created by the method $\texttt{updatepLvlNextFunc}$, which simply creates a trivial $\texttt{pLvlNextFunc}$ attribute with no persistent income dynamics.  This method can be overwritten by subclasses in order to make, e.g., an AR(1) income process. 


* The input $\texttt{pLvlGrid}$ is created by the method $\texttt{updatepLvlGrid}$, which updates the grid of persistent income levels for infinite horizon models ($\texttt{cycles}$=0) and lifecycle models ($\texttt{cycles}$=1). This model (and its children) cannot handle $\texttt{cycles}$>1 because the distribution of persistent income will be different within a period depending on how many cycles have elapsed. This method uses a simulation approach to generate the $\texttt{pLvlGrid}$ at each period of the cycle. It draws on the initial distribution of persistent income, the $\texttt{pLvlNextFunc}$, which is defined by its $\texttt{pLvlInitMean}$ and its $\texttt{pLvlInitStd}$, and the percentiles of the $\texttt{pLvl}$ distribution, the $\texttt{pLvlPctiles}$ attribute. 

### To simulate the distribution of persistent income levels at each period of the lifecycle

| Param | Description | Type | Value|
| :---: | --- | --- | --- | :---: |
|$\texttt{AgentCount}$|Number of agents of this type for the simulation|int|10000|
|$\texttt{pLvlInitMean}$|Initial mean (log) persistent income|float|0.0|
|$\texttt{pLvlInitStd}$|Initial standard deviation (log) persistent income|float|0.0|
|$\texttt{PermShkDstn}$|Distribution of permanent shocks, with a lognormal mean of 1 and<br> standard deviation $\texttt{PermShkStd}$ equal to [0.1]|[np.array]||
|$\texttt{PermShkStd}$||float|[0.1]||


* The input $\texttt{aXtraGrid}$ is created by the method $\texttt{updateAssetsGrid}$ which inherits from $\texttt{IndShockConsumerType}$.


In [ ]:
# This cell defines a dictionary to make an instance of a "generalized income process" consumer.
GenIncDictionary = { 
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Intertemporal discount factor
    "LivPrb" : [0.98],                     #.… Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
# Parameters for constructing the "assets above minimum" grid
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 30,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [0.005,0.01],           # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
# Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1],                  # Standard deviation of log permanent income shocks
    "TranShkStd" : [0.1],                  # Standard deviation of log transitory income shocks
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    "BoroCnstArt" : 0.0,                  # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                  # Use cubic spline interpolation when True, linear interpolation when False
    "vFuncBool" : True,                   # Whether to calculate the value function during solution   
    "cycles": 0,
    "PermGroFac" : [0.0],                  # Permanent income growth factor - not a list but should be as it is in 'time_vary'
    "pLvlPctiles" : np.concatenate(([0.001, 0.005, 0.01, 0.03], np.linspace(0.05, 0.95, num=19),[0.97, 0.99, 0.995, 0.999])),
}

In [ ]:
# Make and solve an example "explicit permanent income" consumer
GenIncExample = GenIncProcessConsumerType(**GenIncDictionary)
GenIncExample.solve()

In [ ]:
# Plot the consumption function
print('Consumption function by pLvl for generalized income consumer:')
pLvlGrid = GenIncExample.pLvlGrid[0]
mLvlGrid = np.linspace(0,20,300)
for p in pLvlGrid:
    M_temp = mLvlGrid + GenIncExample.solution[0].mLvlMin(p)
    C = GenIncExample.solution[0].cFunc(M_temp,p*np.ones_like(M_temp))
    plt.plot(M_temp,C)
plt.xlim(0.,20.)
plt.xlabel('Market resource level mLvl')
plt.ylabel('Consumption level cLvl')
plt.show()

## ExplicitPermIncConsumerType

While $\texttt{ConsGenIndShockModel}$ leaves persistent income dynamics completely unspecified, and the user *could* choose any function s/he wants, some special cases have been pre-specified as subclasses of $\texttt{GenIncProcessConsumerType}$.  Most simply, we could have a *linear* representation of persistent income dynamics.  The model for an $\texttt{IndShockExplicitPermIncConsumerType}$ is mathematically equivalent to the one for an $\texttt{IndShockConsumerType}$, except for its solver **explicitly** tracking permanent income as a state variable.

The agent's problem can be written in Bellman form as:

\begin{eqnarray*}
v_t(M_t,p_t) &=& \max_{C_t} U(C_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [v_{t+1}(M_{t+1}, p_{t+1}) ], \\
A_t &=& M_t - C_t, \\
A_t &\geq& \underline{a}, \\
M_{t+1} &=& R A_t + \theta_{t+1}, \\
p_{t+1} &=& G_{t+1}(p_t)\psi_{t+1}, \qquad G_{t+1}(p_t) = \Gamma_{t+1} p_t, \\
\psi_{t+1},\theta_{t+1} \sim F_{t+1}, &\qquad& \mathbb{E} [F_{\psi t}] = 1, \\
U(C) &=& \frac{C^{1-\rho}}{1-\rho}.
\end{eqnarray*}

This agent type is identical to an $\texttt{IndShockConsumerType}$ but for explicitly tracking $\texttt{pLvl}$ as a state variable during solution as shown in the mathematical representation of $\texttt{ConsGenIndShockModel}$. The only modeling difference from $\texttt{ConsGenIndShockModel}$ model is that the G function has been specified as *linear* -- a permanent income growth factor.

To construct $\texttt{IndShockExplicitPermIncConsumerType}$ as an instance of $\texttt{GenIncProcessConsumerType}$, we need to pass additional parameters to the constructor as shown in the table below.

### Additional parameters to solve IndShockExplicitPermIncConsumerType

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
|(none)|percentiles of the distribution of persistent income|$\texttt{pLvlPctiles}$|||
| $G$ |Expected persistent income next period as a function of current persistent income | $\texttt{pLvlNextFunc}$ | - | $\surd$ |
|$\Gamma $|Permanent income growth factor|$\texttt{PermGroFac}$|[1.0]| |


### Constructed inputs to solve ExplicitPermInc

* In this model, the method $\texttt{updatepLvlNextFunc}$ can be overwritten to create $\texttt{pLvlNextFunc}$ as a sequence of linear functions, indicating constant expected permanent income growth across permanent income levels.  This method uses the attribute $\texttt{PermGroFac}$, and installs a special retirement function when it exists.

In [ ]:
# This cell defines a dictionary to make an instance of "explicit permanent income" consumer.
ExplicitPermIncDictionary = { 
    "CRRA": 2.0,                           # Coefficient of relative risk aversion
    "Rfree": 1.03,                         # Interest factor on assets
    "DiscFac": 0.96,                       # Intertemporal discount factor
    "LivPrb" : [0.98],                     #.… Survival probability
    "AgentCount" : 10000,                  # Number of agents of this type (only matters for simulation)
    "aNrmInitMean" : 0.0,                  # Mean of log initial assets (only matters for simulation)
    "aNrmInitStd"  : 1.0,                  # Standard deviation of log initial assets (only for simulation)
    "pLvlInitMean" : 0.0,                  # Mean of log initial permanent income (only matters for simulation)
    "pLvlInitStd"  : 0.0,                  # Standard deviation of log initial permanent income (only matters for simulation)
    "PermGroFacAgg" : 1.0,                 # Aggregate permanent income growth factor (only matters for simulation)
    "T_age" : None,                        # Age after which simulated agents are automatically killed
    "T_cycle" : 1,                         # Number of periods in the cycle for this agent type
# Parameters for constructing the "assets above minimum" grid
    "aXtraMin" : 0.001,                    # Minimum end-of-period "assets above minimum" value
    "aXtraMax" : 30,                       # Maximum end-of-period "assets above minimum" value               
    "aXtraExtra" : [0.005,0.01],           # Some other value of "assets above minimum" to add to the grid
    "aXtraNestFac" : 3,                    # Exponential nesting factor when constructing "assets above minimum" grid
    "aXtraCount" : 48,                     # Number of points in the grid of "assets above minimum"
# Parameters describing the income process
    "PermShkCount" : 7,                    # Number of points in discrete approximation to permanent income shocks
    "TranShkCount" : 7,                    # Number of points in discrete approximation to transitory income shocks
    "PermShkStd" : [0.1],                  # Standard deviation of log permanent income shocks
    "TranShkStd" : [0.1],                  # Standard deviation of log transitory income shocks
    "UnempPrb" : 0.05,                     # Probability of unemployment while working
    "UnempPrbRet" : 0.005,                 # Probability of "unemployment" while retired
    "IncUnemp" : 0.3,                      # Unemployment benefits replacement rate
    "IncUnempRet" : 0.0,                   # "Unemployment" benefits when retired
    "tax_rate" : 0.0,                      # Flat income tax rate
    "T_retire" : 0,                        # Period of retirement (0 --> no retirement)
    "BoroCnstArt" : 0.0,                  # Artificial borrowing constraint; imposed minimum level of end-of period assets
    "CubicBool" : False,                  # Use cubic spline interpolation when True, linear interpolation when False
    "vFuncBool" : True,                   # Whether to calculate the value function during solution    
# More parameters specific to "Explicit Permanent income" shock model
    "cycles": 0,
    "pLvlPctiles" : np.concatenate(([0.001, 0.005, 0.01, 0.03], np.linspace(0.05, 0.95, num=19),[0.97, 0.99, 0.995, 0.999])),
    "PermGroFac" : [1.0],                  # Permanent income growth factor - long run permanent income growth doesn't work yet    
}

We can now create an instance of the type of consumer we are interested in and solve this agent's problem with an infinite horizon (cycles=0).

In [ ]:
# Make and solve an example of "explicit permanent income" consumer with idiosyncratic shocks
ExplicitExample = IndShockExplicitPermIncConsumerType(**ExplicitPermIncDictionary)
                                        
print('Here, the lowest percentile is ' + str(ExplicitPermIncDictionary['pLvlPctiles'][0]*100))
print('and the highest percentile is ' + str(ExplicitPermIncDictionary['pLvlPctiles'][-1]*100) + '.\n')
      
ExplicitExample.solve()


In the cell below, we generate a plot of the consumption function for an $\texttt{IndShockExplicitPermIncConsumerType}$ at different income levels.

In [ ]:
# Plot the consumption function at various permanent income levels.
print('Consumption function by pLvl for explicit permanent income consumer:')
pLvlGrid = ExplicitExample.pLvlGrid[0]
mLvlGrid = np.linspace(0,20,300)
for p in pLvlGrid:
    M_temp = mLvlGrid + ExplicitExample.solution[0].mLvlMin(p)
    C = ExplicitExample.solution[0].cFunc(M_temp,p*np.ones_like(M_temp))
    plt.plot(M_temp,C)
plt.xlim(0.,20.)
plt.xlabel('Market resource level mLvl')
plt.ylabel('Consumption level cLvl')
plt.show()

## Permanent income normalized

We can compare the model for an $\texttt{IndShockExplicitPermIncConsumerType}$ with an alternative one, in which permanent income is normalized out as in $\texttt{ConsIndShockModel}$. That is, variables such as $M_t$, $C_t$ and $A_t$ are divided by permanent income $p_t$ and we can solve the model as follows.

In [ ]:
# Make and solve an example of normalized model
NormalizedExample = IndShockConsumerType(**ExplicitPermIncDictionary)
NormalizedExample.solve()

In [ ]:
# Compare the normalized problem with and without explicit permanent income and plot the consumption functions
print('Normalized consumption function by pLvl for explicit permanent income consumer:')
pLvlGrid = ExplicitExample.pLvlGrid[0]
mNrmGrid = np.linspace(0,20,300)
for p in pLvlGrid:
    M_temp = mNrmGrid*p + ExplicitExample.solution[0].mLvlMin(p)
    C = ExplicitExample.solution[0].cFunc(M_temp,p*np.ones_like(M_temp))
    plt.plot(M_temp/p,C/p)

plt.xlim(0.,20.)
plt.xlabel('Normalized market resources mNrm')
plt.ylabel('Normalized consumption cNrm')
plt.show()

print('Consumption function for normalized problem (without explicit permanent income):')
mNrmMin = NormalizedExample.solution[0].mNrmMin
plotFuncs(NormalizedExample.solution[0].cFunc,mNrmMin,mNrmMin+20.)

The two figures above show that the normalized consumption function for the "explicit permanent income" consumer is almost identical for every permanent income level (and is the same as the normalized problem's $\texttt{cFunc}$), but is less accurate due to extrapolation outside the bounds of $\texttt{pLvlGrid}$. 

The "explicit permanent income" solution deviates from the solution to the normalized problem because of errors from extrapolating beyond the bounds of the $\texttt{pLvlGrid}$. The error is largest for $\texttt{pLvl}$ values near the upper and lower bounds, and propagates toward the center of the distribution.

In [ ]:
# Plot the value function at various permanent income levels
if ExplicitExample.vFuncBool:
    pGrid = np.linspace(0.1,3.0,24)
    M = np.linspace(0.001,5,300)
    for p in pGrid:
        M_temp = M+ExplicitExample.solution[0].mLvlMin(p)
        C = ExplicitExample.solution[0].vFunc(M_temp,p*np.ones_like(M_temp))
        plt.plot(M_temp,C)
    plt.ylim([-200,0])
    plt.xlabel('Market resource level mLvl')
    plt.ylabel('Value v')
    plt.show()

In [ ]:
# Simulate many periods to get to the stationary distribution
ExplicitExample.T_sim = 500
ExplicitExample.track_vars = ['mLvlNow','cLvlNow','pLvlNow']
ExplicitExample.initializeSim()
ExplicitExample.simulate()
plt.plot(np.mean(ExplicitExample.mLvlNow_hist,axis=1))
plt.xlabel('Simulated time period')
plt.ylabel('Average market resources mLvl')
plt.show()


## 2. Persistent income shock consumer


Finally, we can extends $\texttt{ConsGenIndShockModel}$ by allowing the (log) persistent income to follow an AR(1) process with correlation coefficient $\varphi$. The model for a $\texttt{PersistentShockConsumerType}$ is mathematically equivalent to the one for an $\texttt{IndShockConsumerType}$, except for its solver **explicitly** tracking permanent income as a state variable, and having an **AR(1)** income process rather than random walk.


The agent's problem can be written in Bellman form as:

\begin{eqnarray*}
v_t(M_t,p_t) &=& \max_{C_t} U(C_t) + \beta (1-\mathsf{D}_{t+1}) \mathbb{E} [v_{t+1}(M_{t+1}, p_{t+1}) ], \\
A_t &=& M_t - C_t, \\
A_t &\geq& \underline{a}, \\
M_{t+1} &=& R A_t + \theta_{t+1}, \\
log(p_{t+1}) &=& \varphi log(p_t)+(1-\varphi)log(\overline{p}_{t+1} ) +log(\Gamma_{t+1})+log(\psi_{t+1}), \\
\psi_{t+1},\theta_{t+1} \sim F_{t+1}, &\qquad& \mathbb{E} [F_{\psi t}] = 1 \\
\end{eqnarray*}

### Additional parameters to solve PersistentShock model

| Param | Description | Code | Value | Constructed |
| :---: | --- | --- | --- | :---: |
|$\varphi$|Serial correlation coefficient for persistent income|$\texttt{PrstIncCorr}$|0.98||
|$\Gamma $|Permanent income growth factor|$\texttt{PermGroFac}$|[1.0]| |
|log ($\overline{p}_{t+1}$)|Weighted average of (log) persistent income shocks|$\texttt{pLogMean}$|-||
|(none)|Expected persistent income next period as a function of current persistent income | $\texttt{pLvlNextFunc}$ | - | $\surd$ |

### Constructed inputs to solve PersistentShock

* For this model, the method $\texttt{updatepLvlNextFunc}$ can be overwritten to create the input $\texttt{pLvlNextFunc}$ as a sequence of AR(1)-style functions. The method create a list of $\texttt{pLvlNextFunc}$ by creating an object of an instance of the class $\texttt{pLvlFuncAR1}$ which takes for arguments $\texttt{pLogMean}$, $\texttt{PermGroFac}$ and $\texttt{PrstIncCorr}$. 

We can create a dictionary for a $\texttt{PersistentShockConsumerType}$ as a copy of $\texttt{IndShockExplicitPermIncConsumerType}$ and add the parameter $\texttt{PrstIncCorr}$ to it to specify the AR(1) income process. 

In [ ]:
# This cell defines a dictionary to make an instance of "persistent idiosyncratic shocks" consumer
PrstIncCorr = 0.98       # Serial correlation coefficient for persistent income
persistent_shocks = copy(ExplicitPermIncDictionary)
persistent_shocks['PrstIncCorr'] = PrstIncCorr

In [ ]:
# Make and solve an example of "persistent idiosyncratic shocks" consumer
PersistentExample = PersistentShockConsumerType(**persistent_shocks)
PersistentExample.solve()

In [ ]:
# Plot the consumption function at various levels of persistent income pLvl
print('Consumption function by persistent income level pLvl for a consumer with AR1 coefficient of ' + str(PersistentExample.PrstIncCorr) + ':')
pLvlGrid = PersistentExample.pLvlGrid[0]
mLvlGrid = np.linspace(0,20,300)
for p in pLvlGrid:
    M_temp = mLvlGrid + PersistentExample.solution[0].mLvlMin(p)
    C = PersistentExample.solution[0].cFunc(M_temp,p*np.ones_like(M_temp))
    plt.plot(M_temp,C)
plt.xlim(0.,20.)
plt.xlabel('Market resource level mLvl')
plt.ylabel('Consumption level cLvl')
plt.show()

In [ ]:
# Plot the value function at various persistent income levels
if PersistentExample.vFuncBool:
    pGrid = PersistentExample.pLvlGrid[0]
    M = np.linspace(0.001,5,300)
    for p in pGrid:
        M_temp = M+PersistentExample.solution[0].mLvlMin(p)
        C = PersistentExample.solution[0].vFunc(M_temp,p*np.ones_like(M_temp))
        plt.plot(M_temp,C)
    plt.ylim([-200,0])
    plt.xlabel('Market resource level mLvl')
    plt.ylabel('Value v')
    plt.show()

In [ ]:
# Simulate some data
PersistentExample.T_sim = 500
PersistentExample.track_vars = ['mLvlNow','cLvlNow','pLvlNow']
PersistentExample.initializeSim()
PersistentExample.simulate()
plt.plot(np.mean(PersistentExample.mLvlNow_hist,axis=1))
plt.xlabel('Simulated time period')
plt.ylabel('Average market resources mLvl')
plt.show()